In [2]:
from bot.analysis.pair_trading import backtest
from bot.utils.pair_trading import make_df_from_orderbooks, get_qty, run_single_tf_backtest
from bot.utils.pair_trading import run_double_tf_backtest, run_single_tf_backtest_reverse, select_cols_1tf, select_cols_2tf
from bot.analysis.strategy_analysis import analyze_strategy
from bot.utils.coins import get_step_info, get_price_scale
from bot.utils.files import load_config

from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
import polars as pl
import numpy as np
import pickle
import random
import json
import ast
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['timezone'] = 'Europe/Moscow'

from bot.core.db.postgres_manager import DBManager
from bot.config.credentials import host, user, password, db_name
db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)

In [25]:
config = load_config('./bot/config/config.yaml')

method = config['spr_method']
start_time = config['valid_time']
end_time = config['end_time']

with open("./data/coin_information.pkl", "rb") as f:
    coin_information = pickle.load(f)

token_pairs = []
with open('./data/token_pairs.txt', 'r') as file:
    for line in file:
        a, b = line.strip().split()
        token_pairs.append((a, b))

In [26]:
def check_pos(name):
    token_1, token_2, *_ = name.split('_')
    return any(a == token_1 and b == token_2 for a, b, _ in pairs)

In [27]:
def get_thresholds():
    data = []
    with open('./bot/config/thresholds_test.txt', 'r') as file:
        for line in file:
            line = line.strip()  # Удаляем пробелы и переносы строк
            if line:  # Игнорируем пустые строки
                # Преобразуем строку в кортеж с помощью literal_eval
                tuple_data = ast.literal_eval(line)
                data.append(tuple_data)
    return data

In [28]:
def create_single_tf_backtest_df(token_pairs, start_time, end_time, tf, wind, method):
    time_series = pl.datetime_range(start=start_time, end=end_time, interval="5s", eager=True)
    main_df = pl.DataFrame({'time': time_series})

    for token_1, token_2 in token_pairs:
        try:
            spread_df = pl.read_parquet(f'./data/pair_backtest/{token_1}_{token_2}_{method}_full.parquet',
                low_memory=True, use_pyarrow=True).filter(
                    (pl.col('time') >= start_time) & (pl.col('time') < end_time)
                ).select('time', token_1, f'{token_1}_size', f'{token_1}_bid_price', f'{token_1}_ask_price',
                    f'{token_1}_bid_size', f'{token_1}_ask_size', token_2, f'{token_2}_size',
                    f'{token_2}_bid_price', f'{token_2}_ask_price', f'{token_2}_bid_size', f'{token_2}_ask_size',
                    f'z_score_{wind}_{tf}'
                ).rename({f'z_score_{wind}_{tf}': f'{token_1}_{token_2}_z_score'})

            if token_1 not in main_df.columns:
                df = spread_df.select('time', token_1, f'{token_1}_size', f'{token_1}_bid_price', f'{token_1}_ask_price',
                    f'{token_1}_bid_size', f'{token_1}_ask_size')
                main_df = main_df.join(df, on='time', how='full', coalesce=True)
            if token_2 not in main_df.columns:
                df = spread_df.select('time', token_2, f'{token_2}_size', f'{token_2}_bid_price', f'{token_2}_ask_price',
                    f'{token_2}_bid_size', f'{token_2}_ask_size')
                main_df = main_df.join(df, on='time', how='full', coalesce=True)

            df = spread_df.select('time', f'{token_1}_{token_2}_z_score')
            main_df = main_df.join(df, on='time', how='full', coalesce=True)
        except FileNotFoundError:
            continue

    return main_df

In [29]:
def create_double_tf_backtest_df(token_pairs, start_time, end_time, tf_1, wind_1, tf_2, wind_2):

    time_series = pl.datetime_range(start=start_time, end=end_time, interval="1s", eager=True)
    main_df = pl.DataFrame({'time': time_series})

    for token_1, token_2 in token_pairs:
        try:
            spread_df = pl.scan_parquet(f'./data/pair_backtest/{token_1}_{token_2}_{method}_full.parquet').filter(
                    (pl.col('time') >= start_time) & (pl.col('time') < end_time)
                ).select('time', token_1, f'{token_1}_size', f'{token_1}_bid_price', f'{token_1}_ask_price',
                    f'{token_1}_bid_size', f'{token_1}_ask_size', token_2, f'{token_2}_size',
                    f'{token_2}_bid_price', f'{token_2}_ask_price', f'{token_2}_bid_size', f'{token_2}_ask_size',
                    f'z_score_{wind_1}_{tf_1}', f'z_score_{wind_2}_{tf_2}'
                ).rename({f'z_score_{wind_1}_{tf_1}': f'{token_1}_{token_2}_z_score_1',
                          f'z_score_{wind_2}_{tf_2}': f'{token_1}_{token_2}_z_score_2'}).collect()

            if token_1 not in main_df.columns:
                df = spread_df.select('time', token_1, f'{token_1}_size', f'{token_1}_bid_price', f'{token_1}_ask_price',
                    f'{token_1}_bid_size', f'{token_1}_ask_size')
                main_df = main_df.join(df, on='time')
            if token_2 not in main_df.columns:
                df = spread_df.select('time', token_2, f'{token_2}_size', f'{token_2}_bid_price', f'{token_2}_ask_price',
                    f'{token_2}_bid_size', f'{token_2}_ask_size')
                main_df = main_df.join(df, on='time')

            df = spread_df.select('time', f'{token_1}_{token_2}_z_score_1', f'{token_1}_{token_2}_z_score_2')
            main_df = main_df.join(df, on='time')

        except FileNotFoundError:
            continue

    return main_df

In [30]:
# Загружаем полный датасет
df = pl.scan_parquet('./data/train_data.parquet')

# Выбрасываем столбцы с информацией о спреде, он нам сейчас не нужен
all_cols = df.collect_schema().names()
cols_to_drop = [col for col in all_cols if '_spread_' in col]

df = df.drop(cols_to_drop).collect()
f'{df.estimated_size():_}'

'249_853_583'

#### Поиск наилучших параметров для прямого входа на 1 таймфрейме

In [40]:
metrics_arr = []
in_params = (1.8, 2.0, 2.25, 2.5, 2.75)
out_params = (0.0, 0.25, 0.5)

tfs = ['4h', '1h']
winds = {'4h': [14, 18, 24, 30], '1h': [48, 64, 72, 96, 120]}
leverage = 2
max_pairs = 5
min_order_size = 42
max_order_size = 50
fee_rate = 0.00055
sl_ratio = 0.9

ln = len(in_params) * len(out_params) * len(winds['4h'] + winds['1h'])
ln

135

In [41]:
# Симуляция одного таймфрейма с прямым входом и выходом
all_trades = pl.DataFrame()

with tqdm(total=ln, desc="Обработка", unit="iter") as progress_bar:
    for tf in tfs:
        for wind in winds[tf]:
            tdf = select_cols_1tf(df, token_pairs, tf, wind)
    
            for in_ in in_params:
                for out_ in out_params:
                    trades_df, metrics = run_single_tf_backtest(tdf, tf, wind, in_, out_, leverage, max_pairs, min_order_size, max_order_size,
                               fee_rate, start_time, end_time, sl_ratio, coin_information)
    
                    if trades_df.is_empty():
                        progress_bar.update(1)
                        continue
    
                    all_trades = all_trades.vstack(trades_df)
                    metrics_arr.append({'tf': tf, 'wind': wind, 'thresh_in': in_, 'thresh_out': out_,
                        'n_trades': metrics['n_trades'], 'duration_min': metrics['duration_min'], 'duration_max': metrics['duration_max'],
                        'duration_avg': metrics['duration_avg'], 'stop_losses': metrics['stop_losses'], 'liquidations': metrics['liquidations'],
                        'profit': metrics['profit'], 'max_drawdown': metrics['max_drawdown'], 'max_profit': metrics['max_profit'],
                        'max_loss': metrics['max_loss'], 'avg_profit': metrics['avg_profit'], 'profit_std': metrics['profit_std'],
                        'profit_ratio': metrics['profit_ratio']})
                    progress_bar.update(1)

str_output = pl.DataFrame(metrics_arr).sort(by='profit_ratio', descending=True).drop('duration_max', 'stop_losses', 'liquidations')

Обработка:   0%|          | 0/135 [00:00<?, ?iter/s]

In [42]:
str_output.sort(by='profit_ratio', descending=True)

tf,wind,thresh_in,thresh_out,n_trades,duration_min,duration_avg,profit,max_drawdown,max_profit,max_loss,avg_profit,profit_std,profit_ratio
str,i64,f64,f64,i64,duration[μs],duration[μs],f64,f64,f64,f64,f64,f64,f64
"""1h""",48,2.75,0.5,22,2h 27m 5s,19h 46m 12s,58.25,-1.08,7.98,-1.08,2.65,1.99,10.111
"""4h""",14,2.5,0.5,16,4h 42m 55s,19h 36m 36s,46.13,0.0,6.96,0.0,2.88,1.98,8.874
"""1h""",48,2.75,0.25,22,2h 19m 35s,18h 12m 39s,51.72,-1.47,7.3,-1.47,2.35,1.88,8.692
"""4h""",14,2.5,0.25,16,4h 8m 10s,18h 57m 30s,40.7,-0.25,6.42,-0.25,2.54,1.9,7.649
"""1h""",48,2.75,0.0,23,2h 15m 20s,17h 37m 44s,43.82,-1.69,7.08,-1.69,1.91,1.75,7.244
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""4h""",24,2.0,0.5,7,59m 35s,1d 19h 42m 41s,11.06,-4.78,7.19,-3.1,1.58,3.33,1.404
"""4h""",24,2.5,0.25,7,27m 5s,2d 3h 33m 36s,10.27,-4.1,7.07,-2.48,1.47,3.09,1.372
"""1h""",96,2.75,0.5,4,6h 41m 20s,2d 21h 31m 12s,8.35,-1.74,7.43,-1.34,2.09,3.95,1.318


In [43]:
str_output.group_by('tf', 'wind').agg(
    pl.col('profit').sum().round(2),
    pl.col('profit').min().round(2).alias('min_profit'),
    pl.col('profit').max().round(2).alias('max_profit'),
    (pl.col('profit').sum() / pl.col('profit').len()).round(2).alias('avg_profit'),
    pl.col('profit_ratio').mean().alias('profit_ratio'),
).sort(by='profit_ratio', descending=True).head(10)

tf,wind,profit,min_profit,max_profit,avg_profit,profit_ratio
str,i64,f64,f64,f64,f64,f64
"""4h""",14,504.24,13.68,48.01,33.62,4.95
"""1h""",48,534.23,19.89,58.25,35.62,4.686467
"""1h""",72,533.56,25.15,45.06,35.57,4.451867
"""1h""",64,555.97,27.92,50.0,37.06,3.9764
"""4h""",18,455.52,19.85,39.92,30.37,3.865
"""1h""",120,292.5,11.86,34.8,19.5,2.905667
"""4h""",30,261.32,10.3,34.24,17.42,2.707267
"""1h""",96,275.73,6.8,27.64,18.38,2.359867
"""4h""",24,251.78,9.76,36.18,16.79,2.327933


In [37]:
str_output.sort('profit', descending=True).filter(
    (pl.col('tf') == '1h') & (pl.col('wind') == 120)
)#.head(10)

tf,wind,thresh_in,thresh_out,n_trades,duration_min,duration_avg,profit,max_drawdown,max_profit,max_loss,avg_profit,profit_std,profit_ratio
str,i64,f64,f64,i64,duration[μs],duration[μs],f64,f64,f64,f64,f64,f64,f64
"""1h""",120,2.75,0.0,8,8h 45m 20s,1d 14h 44m 33s,34.8,0.0,8.9,0.0,4.35,2.86,6.585
"""1h""",120,2.0,0.0,10,59m 50s,1d 15h 40m 22s,29.88,-5.39,8.9,-3.49,2.99,3.85,3.606
"""1h""",120,2.25,0.0,8,57m 55s,1d 22h 53m 24s,24.88,-4.69,8.9,-2.79,3.11,4.14,3.128
"""1h""",120,1.8,0.0,10,1h 4m 55s,1d 18h 35m 24s,23.88,-5.6,8.9,-3.7,2.39,3.77,2.847
"""1h""",120,2.5,0.0,7,10h 53m 35s,2d 4h 59m 31s,23.81,-3.53,8.9,-1.98,3.4,4.25,3.244
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""1h""",120,2.25,0.25,7,1h 17m,2d 10h 54m 15s,17.59,-3.5,7.49,-1.95,2.51,3.37,2.441
"""1h""",120,1.8,0.25,7,3h 26m,2d 3h 19m 32s,14.96,-4.43,7.49,-2.89,2.14,3.74,1.929
"""1h""",120,2.0,0.25,6,8h 36m 25s,2d 11h 6m 43s,13.47,-4.21,7.49,-2.66,2.24,4.03,1.754


In [ ]:
# str_output.write_parquet('./data/pair_selection/1tf_direct.parquet')
# str_output = pl.read_parquet('./data/pair_selection/1tf_direct.parquet')

In [ ]:
# 4h"	30	1.8	0.25	22	6h 53m 55s	1d 5h 29m 31s	80.32	0.0	9.82	0.0	3.65	2.59	15.274

In [ ]:
# Симуляция одного таймфрейма со входом и выходом, когда спред возвращается к пороговому значению
metrics_arr = []

dist_in_params = (0.0, 0.1, 0.25, 0.5)
dist_out_params = (0.0, 0.1, 0.25, 0.5)
dist_in = 0
dist_out = 0

ln = len(in_params) * len(out_params) * len(winds['4h'] + winds['1h']) * len(dist_in_params) * len(dist_out_params)

with tqdm(total=ln, desc="Обработка", unit="iter") as progress_bar:
    for tf in tfs:
        for wind in winds[tf]:
            tdf = select_cols_1tf(df, token_pairs, tf, wind)
    
            for in_ in in_params:
                for out_ in out_params:
                    for dist_in in dist_in_params:
                        for dist_out in dist_out_params:
                            if abs(out_) > abs(in_):
                                continue
        
                            trades_df, metrics = run_single_tf_backtest_reverse(tdf, tf, wind, in_, out_, dist_in, dist_out, 
                                    max_pairs, leverage, min_order_size, max_order_size, fee_rate, start_time, end_time,
                                    sl_ratio, coin_information, reverse_in=True, reverse_out=True)
        
                            metrics_arr.append({'tf': tf, 'wind': wind, 'thresh_in': in_, 'thresh_out': out_, 'dist_in': dist_in, 'dist_out': dist_out,
                                'n_trades': metrics['n_trades'], 'duration_min': metrics['duration_min'], 'duration_max': metrics['duration_max'],
                                'duration_avg': metrics['duration_avg'], 'stop_losses': metrics['stop_losses'], 'liquidations': metrics['liquidations'],
                                'profit': metrics['profit'], 'max_drawdown': metrics['max_drawdown'], 'max_profit': metrics['max_profit'],
                                'max_loss': metrics['max_loss'], 'avg_profit': metrics['avg_profit'], 'profit_std': metrics['profit_std'],
                                'profit_ratio': metrics['profit_ratio']})
                            progress_bar.update(1)
rev_output = pl.DataFrame(metrics_arr).sort(by='profit', descending=True).drop('duration_max', 'stop_losses', 'liquidations')

In [ ]:
rev_output.sort(by='profit', descending=True)

In [ ]:
rev_output.group_by('tf', 'wind').agg(
    pl.col('profit').sum().round(2),
    pl.col('profit').min().round(2).alias('min_profit'),
    pl.col('profit').max().round(2).alias('max_profit'),
    (pl.col('profit').sum() / pl.col('profit').len()).round(2).alias('avg_profit'),
    pl.col('profit_ratio').mean().alias('profit_ratio'),
    pl.col('profit').len().alias('n_trades'),
).sort(by='profit_ratio', descending=True).head(10)

In [ ]:
rev_output.sort('profit', descending=True).filter(
    (pl.col('tf') == '4h') & (pl.col('wind') == 24)
)

In [ ]:
# rev_output.write_parquet('./data/pair_selection/1tf_reverse.parquet')

#### Проверка с лучшими параметрами

In [10]:
tf = '1h'
wind = 120
in_ = 2.25
out_ = 0.25
leverage = 2
max_pairs = 5
min_order_size = 42
max_order_size = 50
fee_rate = 0.00055

tdf = select_cols_1tf(df, token_pairs, tf, wind)
trades_df, metrics = run_single_tf_backtest(tdf, tf, wind, in_, out_, leverage, max_pairs, min_order_size, max_order_size,
           fee_rate, start_time, end_time, sl_ratio, coin_information, verbose=True)

2025-11-06 20:46:20+03:00 [long open] buy 4603.0 GMT for 0.0217; sell 2286.0 BLUR for 0.04368; z_score: -2.32
2025-11-06 22:55:40+03:00 [short open] sell 380.9 ARB for 0.2622; buy 630.0 SEI for 0.1585; z_score: 2.26
2025-11-07 01:06:05+03:00 [short open] sell 3347.0 CHZ for 0.02984; buy 6804.0 VET for 0.01468; z_score: 2.27
2025-11-07 01:15:05+03:00 [long open] buy 790.8 IOTA for 0.1263; sell 471.6 MANA for 0.2118; z_score: -2.27
2025-11-07 03:40:10+03:00 [long close] buy 4603.0 GMT for 0.02241; sell 2286.0 BLUR for 0.04294; z_score: 0.28
2025-11-07 03:43:20+03:00 [short open] sell 36.2 DOT for 2.757; buy 50.5 RENDER for 1.977; z_score: 2.28
2025-11-07 09:23:15+03:00 [short open] sell 972.4 MANTA for 0.10272; buy 429.2 CELO for 0.2327; z_score: 2.25
2025-11-07 18:15:25+03:00 [short close] sell 36.2 DOT for 2.8329; buy 50.5 RENDER for 2.144; z_score: -0.25
2025-11-07 18:15:30+03:00 [long open] buy 45.2 XRP for 2.2085; sell 128.9 LDO for 0.7744; z_score: -2.28
2025-11-07 22:04:25+03:00 [

#### Поиск наилучших параметров входа random search

In [ ]:
method = 'dist'
start_time = datetime(2025, 10, 22, 0, 0, tzinfo=ZoneInfo("Europe/Moscow"))
end_time = datetime(2025, 11, 12, 0, 0, tzinfo=ZoneInfo("Europe/Moscow"))

leverage = 2
max_pairs = 5
min_order_size = 45
max_order_size = 50
fee_rate = 0.00055
sl_ratio = 0.45

tfs = ('4h', '1h',)
winds = {'4h': [12, 14, 16, 18, 24, 30],
         '1h': [18, 24, 36, 48, 64, 72, 96, 120],
         #'5m': [60, 90, 120, 180, 240, 300]
        }

n_tf_params = (2, )
in_params = (1.6, 1.8, 2.0, 2.25, 2.5)
out_params = (0.0, 0.25, 0.5,)
dist_in_params = (0.1, 0.25, 0.5)
dist_out_params = (0.1, 0.25, 0.5)

metrics_arr = []

In [ ]:
n_iter = 5_000

with tqdm(total=n_iter, desc="Обработка", unit="iter") as progress_bar:
    for _ in range(n_iter):
        try:
            n_tf = random.choice(n_tf_params)
            trades_df, metrics = None, None

            if n_tf == 1:
                mode = random.choice(['1_tf_direct', '1_tf_rev_in', '1_tf_rev_out', '1_tf_rev_both'])
                tf_1 = random.choice(tfs)
                wind_1 = random.choice(winds[tf_1])
                tf_2, wind_2 = 0, 0
                in_1 = random.choice(in_params)
                out_1 = random.choice(out_params)
                in_2, out_2 = 0, 0

                tdf = select_cols_1tf(df, token_pairs, tf_1, wind_1)

                if mode == '1_tf_direct':
                    dist_in, dist_out = 0, 0
                    trades_df, metrics = run_single_tf_backtest(tdf, tf_1, wind_1, in_1, out_1, leverage, max_pairs, 
                            min_order_size, max_order_size, fee_rate, start_time, end_time, sl_ratio, coin_information)
                    
                elif mode == '1_tf_rev_in':
                    dist_in = random.choice(dist_in_params)
                    dist_out = 0
                    trades_df, metrics = run_single_tf_backtest_reverse(tdf, tf_1, wind_1, in_1, out_1, dist_in, dist_out,  
                                max_pairs, leverage, min_order_size, max_order_size, fee_rate, start_time, end_time,
                                sl_ratio, coin_information, reverse_in=True, reverse_out=False)
                elif mode == '1_tf_rev_out':
                    dist_in = 0
                    dist_out = random.choice(dist_out_params)
                    trades_df, metrics = run_single_tf_backtest_reverse(tdf, tf_1, wind_1, in_1, out_1, dist_in, dist_out,  
                                max_pairs, leverage, min_order_size, max_order_size, fee_rate, start_time, end_time,
                                sl_ratio, coin_information, reverse_in=False, reverse_out=True)
                elif mode == '1_tf_rev_both':
                    dist_in = random.choice(dist_in_params)
                    dist_out = random.choice(dist_out_params)
                    trades_df, metrics = run_single_tf_backtest_reverse(tdf, tf_1, wind_1, in_1, out_1, dist_in, dist_out,  
                                max_pairs, leverage, min_order_size, max_order_size, fee_rate, start_time, end_time,
                                sl_ratio, coin_information, reverse_in=True, reverse_out=True)

            elif n_tf == 2:
                mode = '2_tf_direct'
                tf_1, tf_2 = random.choices(tfs, k=2)
                wind_1 = random.choice(winds[tf_1])
                wind_2 = random.choice(winds[tf_2])
                dist_in, dist_out = 0, 0
                in_1 = random.choice(in_params)
                out_1 = random.choice(out_params)
                in_2 = random.choice(in_params)
                out_2 = random.choice(out_params)

                if tf_1 == tf_2 and wind_1 == wind_2:
                    continue
                if tf_1 == '5m' and tf_2 == '5m':
                    continue

                tdf = select_cols_2tf(df, token_pairs, tf_1=tf_1, wind_1=wind_1, tf_2=tf_2, wind_2=wind_2)
                trades_df, metrics = run_double_tf_backtest(tdf, tf_1, wind_1, tf_2, wind_2, in_1, out_1, in_2, out_2, leverage,
                           max_pairs, min_order_size, max_order_size, fee_rate, start_time, end_time,
                           sl_ratio, coin_information)
            else:
                print('unknown mode!')
                continue

            if not metrics:
                continue

            log = {'n_tf': n_tf, 'tf_1': tf_1, 'tf_2': tf_2, 'wind_1': wind_1, 'wind_2': wind_2,
                    'in_1': in_1, 'in_2': in_2, 'out_1': out_1, 'out_2': out_2,
                    'dist_in': dist_in, 'dist_out': dist_out,
                    'n_trades': metrics['n_trades'],
                    'duration_min': metrics['duration_min'].total_seconds(), 'duration_max': metrics['duration_max'].total_seconds(),
                    'duration_avg': metrics['duration_avg'].total_seconds(), 'stop_losses': metrics['stop_losses'],
                    'liquidations': metrics['liquidations'],
                    'profit': metrics['profit'], 'max_drawdown': metrics['max_drawdown'], 'max_profit': metrics['max_profit'],
                    'max_loss': metrics['max_loss'], 'avg_profit': metrics['avg_profit'], 'profit_std': metrics['profit_std'],
                    'profit_ratio': metrics['profit_ratio']}
            json_log = json.dumps(log, default=float, ensure_ascii=False)
            with open('./logs/backtest_res.jsonl', 'a', encoding='utf-8') as f:
                f.write(json_log + '\n')

            # with open('./logs/trades_bt.jsonl', 'a', encoding='utf-8') as f:
            #     for trade in trades_df.to_dicts():
            #         trade.pop('open_time')
            #         trade.pop('close_time')
            #         trade.pop('duration')

            #         trade_log = json.dumps(trade, default=float, ensure_ascii=False)
            #         f.write(trade_log + '\n')

            if n_tf == 1 and mode[5:] == 'direct':
                                print(f'n_tf: {n_tf} ({mode[5:]:>8}); tf: {tf_1}; wind: {wind_1:>3}; in: {in_1:>4}; \
out: {out_1:>4}; profit: {metrics['profit']:.1f}')

            elif n_tf == 1 and mode[5:] == 'rev_in':
                print(f'n_tf: {n_tf} ({mode[5:]:>8}); tf: {tf_1}; wind: {wind_1:>3}; in: {in_1:>4}; \
out: {out_1:>4}; dist_in: {dist_in:>4}, profit: {metrics['profit']:.1f}')

            elif n_tf == 1 and mode[5:] == 'rev_out':
                print(f'n_tf: {n_tf} ({mode[5:]:>8}); tf: {tf_1}; wind: {wind_1:>3}; in: {in_1:>4}; \
out: {out_1:>4}; dist_out: {dist_out:>4}, profit: {metrics['profit']:.1f}')

            elif n_tf == 1 and mode[5:] == 'rev_out':
                print(f'n_tf: {n_tf} ({mode[5:]:>8}); tf: {tf_1}; wind: {wind_1:>3}; in: {in_1:>4}; \
out: {out_1:>4}; dist_in: {dist_in:>4}, dist_out: {dist_out:>4}, profit: {metrics['profit']:.1f}')

            elif n_tf == 2:
                print(f'n_tf: {n_tf} ({mode[5:]:>8}); tf_1: {tf_1}; wind_1: {wind_1:>3}; tf_2: {tf_2}; \
wind_2: {wind_2:>3}; in_1: {in_1:>4}; out_1: {out_1:>4}; in_2: {in_2:>4}; out_2: {out_2:>4}; profit: {metrics['profit']:.1f}')

            metrics_arr.append(log)
            progress_bar.update(1)
        except Exception as err:
            print(err)
            print(f'{mode=}; {tf_1=}; {wind_1=}; {tf_2=}; {wind_2=}; {in_1=}; {out_1=}; {in_2=}; {out_2=}; {dist_in=}, {dist_out=}')
            continue

In [ ]:
output = pl.DataFrame(metrics_arr).sort(by='profit_ratio', descending=True).drop('duration_min', 'duration_avg',
                            'duration_max', 'stop_losses', 'liquidations')
output

In [ ]:
for row in output[:5].iter_rows(named=True):
    print(row['tf'], row['wind'], row['thresh_in'], row['thresh_out'], row['n_trades'], row['duration_avg'], row['profit'],
         row['max_drawdown'], row['max_profit'], row['max_loss'], row['avg_profit'], row['profit_std'], row['profit_ratio'])

In [ ]:
tf_1 = '1h'
tf_2 = '4h'
wind_1 = 48
wind_2 = 24
in_1 = 2.0
in_2 = 2.0
out_1 = 0.25
out_2 = 0.25

In [ ]:
tdf = select_cols_1tf(df, token_pairs, tf_2, wind_2)
trades_df, metrics = run_single_tf_backtest(tdf, tf_1, wind_1, in_1, out_1, leverage, max_pairs, 
                            min_order_size, max_order_size, fee_rate, start_time, end_time, sl_ratio, coin_information)

In [ ]:
metrics

In [ ]:
tdf = select_cols_2tf(df, token_pairs, tf_1=tf_1, wind_1=wind_1, tf_2=tf_2, wind_2=wind_2)
trades_df, metrics = run_double_tf_backtest(tdf, tf_1, wind_1, tf_2, wind_2, in_1, out_1, in_2, out_2, leverage,
                           max_pairs, min_order_size, max_order_size, fee_rate, start_time, end_time,
                           sl_ratio, coin_information)

In [ ]:
metrics

#### EDA

In [ ]:
from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
import polars as pl
import numpy as np
import pickle
import random
import json
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
mpl.rcParams['timezone'] = 'Europe/Moscow'

In [ ]:
trades = pl.read_ndjson('./logs/trades_bt.jsonl')
trades = trades.unique()
# trades = trades.filter(pl.col('open_z_score') > 0)
# trades = trades.filter((pl.col('open_z_score') > 1.0) & (pl.col('open_z_score') < 5.0))
# start_times = trades['open_ts'].unique().to_list()

# trades = trades.with_columns(
#     (pl.col("total_profit") > 0).alias("profit_cat")
# )

In [ ]:
trades.height

In [ ]:
trades['open_z_score'].min(), trades['open_z_score'].max()

In [ ]:
trades['total_profit'].mean(), trades['total_profit'].median(), trades['total_profit'].min(), trades['total_profit'].max()

In [ ]:
sl_trades = trades.filter(pl.col('reason') != 1).group_by('token_1', 'token_2').agg(pl.col('total_profit').len().alias('n_trades'))

pairs_to_remove = []
for row in sl_trades.iter_rows(named=True):
    pairs_to_remove.append((row['token_1'], row['token_2']))

In [ ]:
pairs_to_remove

In [ ]:
profitable_trades = trades.group_by('token_1', 'token_2').agg(
    pl.col('total_profit').mean().alias('avg_profit'),
    pl.col('total_profit').min().alias('min_profit'),
    pl.col('total_profit').max().alias('max_profit'),
    pl.col('total_profit').len().alias('n_trades'),
    pl.col('reason').mean().alias('mean_reason')
).sort(by='avg_profit').filter(pl.col('avg_profit') > 0.5)

In [ ]:
profitable_trades

In [ ]:
t1_agg_trades = trades.group_by('token_1').agg(
    pl.col('total_profit').sum().alias('profit'),
    pl.col('total_profit').len().alias('n_trades'),
).rename({'token_1': 'token', 'profit': 'profit_1', 'n_trades': 'n_trades_1'})

t2_agg_trades = trades.group_by('token_2').agg(
    pl.col('total_profit').sum().alias('profit'),
    pl.col('total_profit').len().alias('n_trades'),
).rename({'token_2': 'token', 'profit': 'profit_2', 'n_trades': 'n_trades_2'})

In [ ]:
token_df = t1_agg_trades.join(t2_agg_trades, on='token')
token_df = token_df.with_columns(
    ((pl.col('profit_1') + pl.col('profit_2')) / (pl.col('n_trades_1') + pl.col('n_trades_2'))).alias('avg_profit')
).sort(by='avg_profit')

In [ ]:
bad_coins = token_df.filter(pl.col('avg_profit') < 0).select('token').to_series().to_list()

In [ ]:
profitable_trades

In [ ]:
profitable_coins = []
for row in profitable_trades.iter_rows(named=True):
    profitable_coins.append((row['token_1'], row['token_2']))

In [ ]:
# bad_coins = ('C98', 'ARB', 'CELO')

In [ ]:
for token_1, token_2 in profitable_coins:
    if token_1 in bad_coins or token_2 in bad_coins:
        pairs_to_remove.append((token_1, token_2))
pairs_to_remove

In [ ]:
for pair in pairs_to_remove:
    profitable_coins.remove(pair)

In [ ]:
sns.relplot(data=trades, x="open_z_score", y="total_profit", height=4, aspect=3.5);
plt.grid();

In [ ]:
sns.relplot(data=trades, x="spread", y="total_profit", height=4, aspect=3.5);
plt.grid();

In [ ]:
# sns.displot(trades.to_pandas(), x="open_z_score", height=4, aspect=3.5, hue='profit_cat');

In [ ]:
df = pl.scan_parquet('./data/full.parquet')

# Выбрасываем столбцы с информацией о спреде, он нам сейчас не нужен
all_cols = df.collect_schema().names()

In [ ]:
cols = []
add_data = pl.DataFrame()

for row in tqdm(trades.iter_rows(named=True), total=trades.height):
    ts = row['open_ts']
    t1 = row['token_1']
    t2 = row['token_2']
    w = row['wind']
    tf = row['tf']

    col = [col for col in all_cols if (col.startswith(f"{t1}_{t2}") and col.endswith(f"_{w}_{tf}") and 'spread' in col)]
    cols.extend(col)

In [ ]:
cols = list(set(cols))

In [ ]:
df = df.filter(pl.col('ts').is_in(start_times)).select(['time', 'ts'] + cols).collect()

In [ ]:
add_data = pl.DataFrame()
spr, spr_mean, spr_std = [], [], []

for row in tqdm(trades.iter_rows(named=True), total = trades.height):
    ts = row['open_ts']
    t1 = row['token_1']
    t2 = row['token_2']
    w = row['wind']
    tf = row['tf']

    col = ['ts'] + [col for col in all_cols if (col.startswith(f"{t1}_{t2}") and col.endswith(f"_{w}_{tf}") and 'spread' in col)]
    tr = df.select(col).filter(pl.col('ts') == ts)

    try:
        spr.append(tr[f'{t1}_{t2}_spread_{w}_{tf}'][0])
        spr_mean.append(tr[f'{t1}_{t2}_spread_mean_{w}_{tf}'][0])
        spr_std.append(tr[f'{t1}_{t2}_spread_std_{w}_{tf}'][0])
    except IndexError:
        spr.append(None)
        spr_mean.append(None)
        spr_std.append(None)

In [ ]:
# trades = trades.with_columns(
#     pl.Series(spr).alias('spread'),
#     pl.Series(spr_mean).alias('spread_mean'),
#     pl.Series(spr_std).alias('spread_std'),
# )

# trades.write_parquet('./data/trades.parquet')

In [ ]:
trades

In [ ]:
trades.drop('close_ts')

In [ ]:
trades['spread_mean'].min(), trades['spread_mean'].max()

In [ ]:
trades.filter(pl.col('side') == 'long').drop('token_1', 'token_2', 'side', 'tf').corr()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

import pandas as pd

In [ ]:
X = trades.drop("total_profit", 'token_1', 'token_2', 'open_ts', 'close_ts').to_pandas()
y = trades["total_profit"].to_pandas()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.2, random_state=42)
rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X_train, y_train)

res = permutation_importance(rf, X_test, y_test, n_repeats=30, random_state=42)
imp = pd.Series(res.importances_mean, index=X.columns).sort_values(ascending=False)
print(imp.head(20))

In [ ]:
# stats = pl.read_parquet('./data/trades_stats.parquet')
stats = pl.read_ndjson('./logs/backtest_res.jsonl')
stats = stats.with_columns(
    (abs(pl.col('in_1')) + abs(pl.col('out_1'))).alias('dist_1'),
    (abs(pl.col('in_2')) + abs(pl.col('out_2'))).alias('dist_2')
)
stats = stats.drop('duration_min', 'duration_max', 'duration_avg', 'liquidations', 'max_profit')
stats.height

In [ ]:
stats.sort(by='profit_ratio', descending=True).head(10)

In [ ]:
stats.group_by('tf_1', 'tf_2', 'wind_1', 'wind_2').agg(
    pl.col('profit').mean().alias('mean_run'),
    pl.col('profit').max().alias('best_run'),
    pl.col('profit').min().alias('worst_run'),
    pl.col('max_drawdown').mean().alias('mean_drdn'),
    pl.col('profit').len().alias('n_trades')
).sort(by='mean_run', descending=True).head(10)

In [ ]:
n_iters = 10

tf_1 = '4h'
tf_2 = '5m'
wind_1 = 18
wind_2 = 60

in_1 = 2.0
in_2 = 2.0
out_1 = 0.25
out_2 = 0.25

leverage = 2
sl_ratio = 0.1
max_pairs = 5
min_order_size = 40
max_order_size = 50
fee_rate = 0.00055

start_time = datetime(2025, 9, 16, 0, tzinfo=ZoneInfo("Europe/Moscow"))
end_time = datetime(2025, 9, 26, 0, tzinfo=ZoneInfo("Europe/Moscow"))

# tdf = select_cols_2tf(df, profitable_coins, tf_1=tf_1, wind_1=wind_1, tf_2=tf_2, wind_2=wind_2)
tdf = select_cols_1tf(df, profitable_coins, tf_1, wind_1)

profit_arr = []
profit_ratio_arr = []
max_drawdown_arr = []
n_trades_arr = []

for i in tqdm(range(n_iters)):
    # trades_df, metrics = run_double_tf_backtest(tdf, tf_1, wind_1,
    #                                 tf_2, wind_2, in_1, out_1, in_2, out_2, leverage,
    #                                 max_pairs, max_order_size, fee_rate, start_time, end_time,
    #                                 coin_information)
    trades_df, metrics = run_single_tf_backtest(tdf, tf_1, wind_1, in_1, out_1, leverage, max_pairs, min_order_size, max_order_size,
                           fee_rate, start_time, end_time, sl_ratio, coin_information=coin_information)

    profit_arr.append(metrics['profit'])
    profit_ratio_arr.append(metrics['profit_ratio'])
    max_drawdown_arr.append(metrics['max_drawdown'])
    n_trades_arr.append(metrics['n_trades'])

mean_profit = sum(profit_arr) / n_iters
mean_pr_ratio = sum(profit_ratio_arr) / n_iters
mean_drdn = sum(max_drawdown_arr) / n_iters
mean_n_tr = int(sum(n_trades_arr) / n_iters)

print(f'mean profit: {mean_profit:.1f}; mean pr_ratio: {mean_pr_ratio:.1f}; mean drdwn: {mean_drdn:.1f}; mean n_trades: {mean_n_tr}')

In [ ]:
# 6.68 sec / it

In [ ]:
# tf_1 = '5m', tf_2 = '1h', wind_1 = 60, wind_2 = 72, in_1 = 2.0, in_2 = 2.0, out_1 = 0.25, out_2 = 0.25
# train: mean profit: 40.9; mean pr_ratio: 3.8; mean drdwn: -11.3; mean n_trades: 56
# test:  mean profit: -3.6; mean pr_ratio: -0.1; mean drdwn: -38.5; mean n_trades: 42

# in_1 = 2.25, in_2 = 2.25,
# train: mean profit: 32.2; mean pr_ratio: 2.9; mean drdwn: -10.9; mean n_trades: 47
# test:  mean profit: -51.5; mean pr_ratio: -1.2; mean drdwn: -63.4; mean n_trades: 41

# tf_1 = '4h', tf_2 = '5m', wind_1 = 18, wind_2 = 180, in_1 = 2.25, in_2 = 2.25, out_1 = 0.25, out_2 = 0.25
# train: mean profit: 40.7; mean pr_ratio: 6.0; mean drdwn: -3.1; mean n_trades: 50
# test:  mean profit: -67.4; mean pr_ratio: -1.2; mean drdwn: -84.6; mean n_trades: 36

# in_1 = 2.0, in_2 = 2.0,
# train: mean profit: 43.1; mean pr_ratio: 3.7; mean drdwn: -12.0; mean n_trades: 61
# test:  mean profit: -66.4; mean pr_ratio: -1.3; mean drdwn: -80.0; mean n_trades: 40

# tf_1 = '4h', tf_2 = '5m', wind_1 = 18, wind_2 = 180, in_1 = 2.25, in_2 = 2.25, out_1 = 2.0, out_2 = 0.25
# train: mean profit: 11.0; mean pr_ratio: 1.2; mean drdwn: -8.2; mean n_trades: 34
# test:  mean profit: -45.4; mean pr_ratio: -1.1; mean drdwn: -62.3; mean n_trades: 32

# tf_1 = '4h', wind_1 = 18, in_1 = 2.0, out_1 = 0.25
# train: mean profit: 24.0; mean pr_ratio: 2.2; mean drdwn: -10.6; mean n_trades: 58
# test:  mean profit: -44.1; mean pr_ratio: -0.8; mean drdwn: -70.4; mean n_trades: 42
# test:  mean profit: 29.1; mean pr_ratio: 1.5; mean drdwn: -26.6; mean n_trades: 46 (only profitable pairs)

In [ ]:
trades_df.filter(pl.col('total_profit') > 0).height, trades_df.filter(pl.col('total_profit') < 0).height

In [ ]:
trades_df.drop('beta', 'fees').sort(by='total_profit').head(5)

In [ ]:
from bot.utils.pair_trading import make_df_from_orderbooks, make_trunc_df, make_zscore_df
from bot.utils.pair_trading import create_zscore_df, get_lr_zscore, get_qty, round_down, calculate_profit_curve, get_thresholds

In [ ]:
1974.1 * 0.05 + 4274.0 * 0.023

In [ ]:
token_1 = 'C98'
token_2 = 'VET'
qty_1 = 1974.1
qty_2 = 4274.0
side = 'short'


min_order = 40
tf = '4h'
wind = 18
winds = np.array([wind,])

train_time = datetime(2025, 9, 24, 8, 14, tzinfo=ZoneInfo("Europe/Moscow"))
st_t = datetime(2025, 10, 2, 8, 14, 30, tzinfo=ZoneInfo("Europe/Moscow"))
end_t = datetime(2025, 10, 2, 11, 15, 30, tzinfo=ZoneInfo("Europe/Moscow"))
start_ts = int(datetime.timestamp(st_t))
median_length = 4

df_1 = db_manager.get_tick_ob(token=token_1 + '_USDT',
                                     start_time=train_time,
                                     end_time=end_t)
df_2 = db_manager.get_tick_ob(token=token_2 + '_USDT',
                                     start_time=train_time,
                                     end_time=end_t)

avg_df = make_df_from_orderbooks(df_1, df_2, token_1, token_2, start_time=train_time)
agg_df = make_trunc_df(avg_df, timeframe='4h', token_1=token_1, token_2=token_2, method='triple', offset='3h')

tick_df = make_df_from_orderbooks(df_1, df_2, token_1, token_2, start_time=st_t)
spread_df = create_zscore_df(token_1, token_2, tick_df, agg_df, tf, winds, min_order, start_ts, median_length)

t1_op = tick_df[f'{token_1}_ask_price'][0] if side == 'long' else tick_df[f'{token_1}_bid_price'][0]
t2_op = tick_df[f'{token_2}_bid_price'][0] if side == 'long' else tick_df[f'{token_2}_ask_price'][0]

res_df = spread_df.select('time', 'ts', token_1, token_2, f'{token_1}_size', f'{token_2}_size',
     f'{token_1}_bid_price', f'{token_1}_ask_price', f'{token_1}_bid_size', f'{token_1}_ask_size',
     f'{token_2}_bid_price', f'{token_2}_ask_price', f'{token_2}_bid_size', f'{token_2}_ask_size',
     f'z_score_{wind}_{tf}').filter(
        (pl.col('time') >= st_t) & (pl.col('time') <= end_t)
     ).rename({f'z_score_{wind}_{tf}': 'z_score'})
profit_df = calculate_profit_curve(res_df, token_1, token_2, side, t1_op, t2_op, t1_qty=qty_1, t2_qty=qty_2, fee_rate=fee_rate)

In [ ]:
fig, (ax1, ax3) = plt.subplots(2, 1, figsize=(14, 6), sharex=True)
# --- Графики цены ---
ax1.plot(tick_df['time'], tick_df[token_1], label=token_1, color='orange')
ax1.set_ylabel('udst')
ax1.legend()
ax1.grid()

ax2 = ax1.twinx()
ax2.plot(tick_df['time'], tick_df[token_2], label=token_2);

ax3.plot(res_df['time'], res_df['z_score'])
ax3.grid()

ax4 = ax3.twinx()
ax4.plot(profit_df['time'], profit_df['profit'], c='g')

plt.legend();

In [ ]:
profit_df

In [ ]:
metrics

In [ ]:
{'total_days': 10.0,
 'n_trades': 55,
 'duration_min': datetime.timedelta(seconds=185),
 'duration_max': datetime.timedelta(days=2, seconds=38660),
 'duration_avg': datetime.timedelta(seconds=67501),
 'stop_losses': 0,
 'liquidations': 0,
 'initial_balance': 200.0,
 'final_balance': 261.3689,
 'profit': 61.37,
 'total_perc_return': 30.68,
 'max_drawdown': -5.14,
 'max_profit': 15.18,
 'max_loss': -4.76,
 'avg_profit': 1.12,
 'profit_std': 2.96,
 'profit_ratio': 7.654}


In [ ]:
trades = pl.read_ndjson('./logs/trades_bt.jsonl')


In [ ]:
trades.group_by('token_1', 'token_2').agg(
    pl.col('total_profit').mean().alias('mean_run'),
    pl.col('total_profit').max().alias('best_run'),
    pl.col('total_profit').min().alias('worst_run'),
    pl.col('total_profit').len().alias('n_trades')
).sort(by='mean_run', descending=True)#.filter((pl.col('token_1') == 'IOTA') & (pl.col('token_2') == 'SAND'))

In [ ]:
# for row in trades.group_by('token_1', 'token_2').agg().iter_rows(named=True):
#     print(row['token_1'], row['token_2'])

In [ ]:
orders = pl.read_ndjson('./logs/trades.jsonl')

token_1 = 'APT'
token_2 = 'FIL'

ot = "2025-09-16 17:07:14"
ct = "2025-09-16 23:09:17"

open_ = orders.filter((pl.col('token_1') == token_1) & (pl.col('token_2') == token_2) & (pl.col('action') == 'open') & (pl.col('ct') == ot))
close = orders.filter((pl.col('token_1') == token_1) & (pl.col('token_2') == token_2) & (pl.col('action') == 'close') & (pl.col('ct') == ct))

In [ ]:
open_.select('ct', 'token_1', 'token_2', 'tf', 'wind', 'thresh_in', 'thresh_out', 'side', 'action',
            't1_bid_price', 't1_ask_price', 't2_bid_price', 't2_ask_price', 'z_score', 'beta')

In [ ]:
close.select('ct', 'token_1', 'token_2', 'tf', 'wind', 'thresh_in', 'thresh_out', 'side', 'action',
            't1_bid_price', 't1_ask_price', 't2_bid_price', 't2_ask_price', 'z_score', 'beta')

In [ ]:
max_position_size = 200
leverage = 2
tf = open_['tf'][0]
wind = open_['wind'][0]

side = open_['side'][0]
side_2 = 'short' if side == 'long' else 'long'
beta = open_['beta'][0]
t1_open = open_['t1'].to_numpy()[0]
t2_open = open_['t2'].to_numpy()[0]
t1_close = close['t1'].to_numpy()[0]
t2_close = close['t2'].to_numpy()[0]

t1_op = open_['t1_ask_price'][0] if side == 'long' else open_['t1_bid_price'][0]
t2_op = open_['t2_bid_price'][0] if side == 'long' else open_['t2_ask_price'][0]
t1_cl = close['t1_bid_price'][0] if side == 'long' else close['t1_ask_price'][0]
t2_cl = close['t2_ask_price'][0] if side == 'long' else close['t2_bid_price'][0]

In [ ]:
make_zscore_df(pl.DataFrame({token_1: t1_open, token_2: t2_open}), token_1, token_2, wind, method='lr').tail(2)

In [ ]:
train_len = 80
wind = 10
winds = np.array((wind,))
tf = '4h'
median_length = 6
min_order = 50

train_time = datetime(2025, 9, 12, 17, 0, tzinfo=ZoneInfo("Europe/Moscow"))
start_time = datetime(2025, 9, 16, 17, 7, 14, tzinfo=ZoneInfo("Europe/Moscow"))
end_time = datetime(2025, 9, 16, 23, 9, 17, tzinfo=ZoneInfo("Europe/Moscow"))

start_ts = int(datetime.timestamp(start_time))

df_1 = db_manager.get_tick_ob(token=token_1 + '_USDT',
                                 start_time=train_time,
                                 end_time=end_time)
df_2 = db_manager.get_tick_ob(token=token_2 + '_USDT',
                                 start_time=train_time,
                                 end_time=end_time)

avg_df = make_df_from_orderbooks(df_1, df_2, token_1, token_2, start_time=train_time)
agg_df = make_trunc_df(avg_df, timeframe='4h', token_1=token_1, token_2=token_2, method='triple', offset='3h')

df_sec = make_trunc_df(avg_df, timeframe='1s', token_1=token_1, token_2=token_2,
                       start_date=start_time - timedelta(seconds=6),
                       end_date = end_time + timedelta(seconds=6),
                       method='last', return_bid_ask=True)
spread_df = create_zscore_df(token_1, token_2, df_sec, agg_df, winds, min_order, start_ts, median_length)

df = spread_df.select('time', 'ts', token_1, token_2, f'{token_1}_size', f'{token_2}_size',
     f'{token_1}_bid_price', f'{token_1}_ask_price', f'{token_1}_bid_size', f'{token_1}_ask_size',
     f'{token_2}_bid_price', f'{token_2}_ask_price', f'{token_2}_bid_size', f'{token_2}_ask_size',
     'z_score').filter(
        (pl.col('time') >= start_time) & (pl.col('time') <= end_time)
     )

In [ ]:
spread_df = create_zscore_df(token_1, token_2, df_sec, agg_df, winds, min_order, start_ts, median_length)


In [ ]:
t1_op, t1_cl

In [ ]:
t2_op, t2_cl

In [ ]:
qty_1, qty_2 = get_qty(token_1, token_2, t1_op, t2_op, beta, coin_information, max_position_size * leverage,
                          method='usdt_neutral')
qty_1, qty_2

In [ ]:
pr_1 = calculate_profit(open_price=t1_op, close_price=t1_cl, n_coins=qty_1, side=side)
pr_2 = calculate_profit(open_price=t2_op, close_price=t2_cl, n_coins=qty_2, side=side_2)
pr_1, pr_2, pr_1 + pr_2

In [ ]:
t1_op, t1_cp

In [ ]:
make_zscore_df(pl.DataFrame({token_1: t1_close, token_2: t2_close}), token_1, token_2, wind, method='lr').tail(1)

In [ ]:
qty_1 = 10881
qty_2 = 4375

open_spread = -0.000066
close_spread = 0.000021
open_mean = 0.000015
open_std = 0.00005

t1_bid_ask_spread = open_['t1_ask_price'][0] - open_['t1_bid_price'][0]
t2_bid_ask_spread = open_['t2_ask_price'][0] - open_['t2_bid_price'][0]
beta = open_['beta'][0]
z_score = open_['z_score'][0]
fee_rate = 0.00055

In [ ]:
# Доход за 1 стандартное отклонение
profit_per_std = qty_1 * open_std
profit_per_std

In [ ]:
# Считаем профит
spread_profit = abs(open_spread - (open_mean + 0.5 * open_std))
real_spread_profit = (close_spread - open_spread) * qty_1
spread_profit * qty_1, real_spread_profit

In [ ]:
# Комиссия за 4 сделки, каждая на 200$ (100$ с плечом 2)
fees = 4 * 200 * fee_rate

# Bid-ask spread
bid_ask_slippage = t1_bid_ask_spread * qty_1 + t2_bid_ask_spread * qty_2
fees, bid_ask_slippage